# 100% local Agentic RAG

In [1]:
import warnings
from crewai import Agent, Crew, Task, LLM, Process
from src.agentic_rag.tools.custom_tool import DocumentSearchTool
# from src.agentic_rag.tools.custom_tool import FireCrawlWebSearchTool

warnings.filterwarnings("ignore")

### Setup LLM


In [2]:
llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

### Setup Tools

In [3]:
pdf_tool = DocumentSearchTool(file_path='./knowledge/dspy.pdf')

Make sure you have the API key for FireCrawl in your environment variables.

In [4]:
# web_search_tool = FireCrawlWebSearchTool()

### Agents

In [5]:
retriever_agent = Agent(
    role="""Retrieve relevant information to answer the user query: {query}""",
    goal="""Retrieve the most relevant information from the available sources 
            for the user query: {query}, always try to use the pdf search tool first. 
            If you are not able to retrieve the information from the pdf search tool 
            then try to use the web search tool.""",
    backstory="""You're a meticulous analyst with a keen eye for detail. 
                You're known for your ability understand the user query: {query} 
                and retrieve knowlege from the most suitable knowledge base.""",
    verbose=True,
    tools=[
        pdf_tool,
        # web_search_tool
    ],
    llm=llm
)

response_synthesizer_agent = Agent(
    role="""Response synthesizer agent for the user query: {query}""",
    goal="""Synthesize the retrieved information into a concise and coherent response 
            based on the user query: {query}. If you are not able to retrieve the 
            information then respond with "I'm sorry, I couldn't find the information 
            you're looking for.""",
    backstory="""You're a skilled communicator with a knack for turning complex 
                information into clear and concise responses.""",
    verbose=True,
    llm=llm
)


### Tasks

In [6]:
retrieval_task = Task(
    description="""Retrieve the most relevant information from the available 
                   sources for the user query: {query}""",
    expected_output="""The most relevant information in form of text as retrieved
                       from the sources.""",
    agent=retriever_agent
)

response_task = Task(
    description="""Synthesize the final response for the user query: {query}""",
    expected_output="""A concise and coherent response based on the retrieved infromation
                       from the right source for the user query: {query}. If you are not 
                       able to retrieve the information then respond with 
                       I'm sorry, I couldn't find the information you're looking for.""",
    agent=response_synthesizer_agent
)

### Initialize Crew

In [7]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

### Kickoff Crew

In [8]:
# Corretta struttura degli input
inputs = {"query": "what exactly is dspy?"}

In [9]:
result = crew.kickoff(inputs=inputs)

# Agent: Retrieve relevant information to answer the user query: what exactly is dspy?
## Task: Retrieve the most relevant information from the available 
                   sources for the user query: what exactly is dspy?


# Agent: Retrieve relevant information to answer the user query: what exactly is dspy?
## Thought: Thought: I need to retrieve relevant information about "dspy" from available sources. First, I will try using the DocumentSearchTool to search for PDF documents containing the query.
## Using tool: DocumentSearchTool
## Tool Input: 
"{\"query\": \"dspy\"}"
## Tool Output: 
, implementations of popular and reusable pipelines (e.g., particular
agents and specific retrieval pipelines) and tools (e.g., connections to various databases and imple-
mentations of long- and short-term memory for agents).

These off-the-shelf higher-level abstractions contrast with DSPy’s focus on introducing core com-
posable operators.
In particular, DSPy introduces signatures (to abstract p

In [10]:
print(result)

Dspy is an open-source debugging tool designed for JavaScript applications. It allows developers to inspect and modify variables, functions, and expressions in real-time, making it easier to identify and fix issues with complex codebases. Dspy provides a graphical interface for debugging, allowing users to visualize the execution of their code and understand how different variables interact with each other. This makes it an invaluable tool for developers looking to improve the performance, reliability, and maintainability of their JavaScript applications.
